In [23]:
%load_ext autoreload
%autoreload 2
import time
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from pprint import pprint

from utils import *
from market_data import *

from indicators import *

import my_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
print(ccxt.__version__)

4.4.90


In [25]:
bitget = create_bitget_exchange(sandbox=True)
is_in_position(exchange=bitget, symbol="ETHUSDT")

False

In [18]:
st = time.time()
price_cache_btc = fetch_latest_ohlcv_to_deque("BTCUSDT", bitget, "1m", maxlen=1000, only_close=False)
price_cache_eth =  fetch_latest_ohlcv_to_deque("ETHUSDT", bitget, "1m", maxlen=1000, only_close=False)
print(time.time() - st)

0.5843942165374756


In [19]:
st = time.time()
close_btc = get_last_closed_ohlcv(bitget, "BTCUSDT", "1m", only_close=False)
close_alt = get_last_closed_ohlcv(bitget, "BTCUSDT", "1m", only_close=False)
print(time.time() - st)

0.5133647918701172


In [ ]:
def process_strategie(pair, 
                      exchange, 
                      timeframe="1m", 
                      maxlen=1000, 
                      pair2=None, 
                      leverage=3):
    
    exchange.set_leverage(leverage, pair2, params={"marginMode": "isolated"})

    price_cache_btc = fetch_latest_ohlcv_to_deque(pair, exchange, timeframe, maxlen=maxlen, only_close=False)
    price_cache_eth =  fetch_latest_ohlcv_to_deque(pair2, exchange, timeframe, maxlen=maxlen, only_close=False)

    # in_position = is_in_position(exchange=exchange, symbol=pair2)
    wait_for_next_candle(timeframe=timeframe)

    in_position = False
    while True:
        try:
                print("time : ", datetime.now())
                close_btc = get_last_closed_ohlcv(exchange, pair, timeframe, only_close=False)
                close_alt = get_last_closed_ohlcv(exchange, pair2, timeframe, only_close=False)

                price_cache_btc.append(close_btc)
                price_cache_eth.append(close_alt)

                data_btc = pd.DataFrame(price_cache_btc, columns=["date", "open", "high", "low", "close", "volume"]).set_index("date")
                data_eth = pd.DataFrame(price_cache_eth, columns=["date", "open", "high", "low", "close", "volume"]).set_index("date")


                # TO DO GET CMMA SIGNAL
                diff, sig = get_cmma_signal( data_btc,
                                             data_eth, 
                                             lookback=my_config.SMA_LOOKBACK, 
                                             atr_lookback=my_config.ATR_LOOKBACK, 
                                             entry_threshold= my_config.ENTRY_THRESHOLD, 
                                             exit_threshold=my_config.EXIT_THRESHOLD)
                
                print("📈 Dernier intermarket_diff :", diff)
                print("Dernier  signal : ", sig)

                if not in_position and sig == 1:
                    print("🟢 Signal d'entrée long détecté")
                    order = bitget.create_market_buy_order(pair2, 0.01, {"marginMode":"isolated"})
                    in_position = True
                    print("✅ Entrée LONG:", order)

                elif in_position and sig == -1:
                    print("🔴 Signal de sortie détecté")
                    order = bitget.create_market_sell_order(pair2, 0.01, {"reduceOnly":True, 
                                                                          "marginMode":"isolated"})
                    in_position = False
                    print("✅ Sortie LONG:", order)

                 # Attente jusqu'à la prochaine bougie
                wait_for_next_candle(timeframe=timeframe)


        except Exception as e:
            print(e)
            return

data_btc, data_eth, diff = process_strategie(pair="BTCUSDT", exchange=bitget, timeframe="1m", maxlen=800, pair2="ETHUSDT")

deque([{'date': Timestamp('2025-06-21 23:53:00'), 'open': 101407.0, 'high': 101413.5, 'low': 101360.0, 'close': 101360.0, 'volume': 64.761}, {'date': Timestamp('2025-06-21 23:54:00'), 'open': 101360.0, 'high': 101624.5, 'low': 101360.0, 'close': 101624.5, 'volume': 59.12}, {'date': Timestamp('2025-06-21 23:55:00'), 'open': 101624.5, 'high': 101732.5, 'low': 101624.5, 'close': 101732.5, 'volume': 57.208}, {'date': Timestamp('2025-06-21 23:56:00'), 'open': 101732.5, 'high': 101808.5, 'low': 101718.5, 'close': 101792.0, 'volume': 67.8641}, {'date': Timestamp('2025-06-21 23:57:00'), 'open': 101792.0, 'high': 102220.0, 'low': 101792.0, 'close': 102220.0, 'volume': 31.1261}, {'date': Timestamp('2025-06-21 23:58:00'), 'open': 102220.0, 'high': 102220.0, 'low': 102046.0, 'close': 102106.0, 'volume': 46.854}, {'date': Timestamp('2025-06-21 23:59:00'), 'open': 102106.0, 'high': 102113.5, 'low': 102049.8, 'close': 102080.0, 'volume': 49.942}, {'date': Timestamp('2025-06-22 00:00:00'), 'open': 102